## This file was used to debug the model, should be ignored

In [1]:
import pytorch_lightning as pl
import torch.nn as nn
import torch
import torch.optim as optim

In [2]:
class CNN_block(nn.Module):
    """
    One CNN block consists of a 1D (3) convolution, a Max pooling and a Batch normalization
    """
    def __init__(self, input_size, kernel_size, in_channels, out_channels):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(in_channels=in_channels,
                      out_channels=out_channels,
                      kernel_size=kernel_size,
                      padding="same"),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )

    def forward(self, x):
        return self.net(x)

In [3]:
def create_model(model_name, model_hparams):
    model = nn.Sequential(
        CNN_block(3000, 3, 1, 32),
        CNN_block(1500, 3, 32, 64),
        CNN_block(750, 3, 64, 64),
        nn.Flatten(),
        nn.Linear(in_features=375*64, out_features=256),
        nn.ReLU(),
        nn.Linear(in_features=256, out_features=5)
    )
    return model

In [4]:
class CNNmodel(pl.LightningModule):
    def __init__(self, model_name, model_hparams, optimizer_name, optimizer_hparams):
        super().__init__()
        # Exports the hyperparameters to a YAML file, and create "self.hparams" namespace
        self.save_hyperparameters()
        self.loss_module = nn.CrossEntropyLoss()

        # Create model
        self.model = create_model(model_name, model_hparams)

        # Example input for visualizing the graph in Tensorboard
        self.example_input_array = torch.zeros((1, 3000), dtype=torch.float32)

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        # We will support Adam or SGD as optimizers.
        if self.hparams.optimizer_name == "Adam":
            # AdamW is Adam with a correct implementation of weight decay (see here for details: https://arxiv.org/pdf/1711.05101.pdf)
            optimizer = optim.AdamW(
                self.parameters(), **self.hparams.optimizer_hparams)
        elif self.hparams.optimizer_name == "SGD":
            optimizer = optim.SGD(self.parameters(), **self.hparams.optimizer_hparams)
        else:
            assert False, f"Unknown optimizer: \"{self.hparams.optimizer_name}\""

        # We will reduce the learning rate by 0.1 after 100 and 150 epochs
        scheduler = optim.lr_scheduler.MultiStepLR(
            optimizer, milestones=[100, 150], gamma=0.1)
        return [optimizer], [scheduler]

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        preds = self.model(inputs)
        loss = self.loss_module(preds, labels)
        acc = (preds.argmax(dim=-1) == labels).float().mean()

        # Logs the accuracy per epoch to tensorboard (weighted average over batches)
        self.log('train_acc', acc, on_step=False, on_epoch=True)
        self.log('train_loss', loss)
        return loss  # Return tensor to call ".backward" on

    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        preds = self.model(inputs)
        acc = (labels == preds.argmax(dim=-1)).float().mean()
        self.log('val_acc', acc)

    def test_step(self, batch, batch_idx):
        inputs, labels = batch
        preds = self.model(inputs)
        acc = (labels == preds.argmax(dim=-1)).float().mean()
        self.log('test_acc', acc)

In [5]:
from data import EEGdataModule
dm = EEGdataModule('../../data/')
dm.setup()
batch, labels = next(iter(dm.train_dataloader()))
batch.shape

torch.Size([64, 1, 3000])

In [6]:
net = CNNmodel(model_name="1D_CNN",
                 model_hparams={},
                 optimizer_name="Adam",
                 optimizer_hparams={
                     "lr": 1e-3,
                     "weight_decay": 1e-4
                 })


In [7]:
preds = net(batch)
print(labels)

tensor([2., 2., 2., 4., 0., 2., 2., 0., 0., 0., 1., 2., 4., 2., 0., 2., 0., 2.,
        0., 2., 0., 2., 2., 2., 3., 0., 0., 3., 2., 3., 2., 2., 0., 0., 2., 4.,
        2., 0., 3., 2., 0., 2., 0., 2., 2., 3., 2., 0., 2., 0., 2., 2., 0., 0.,
        0., 4., 0., 4., 3., 0., 2., 2., 0., 0.])


In [8]:
loss = nn.CrossEntropyLoss()

loss(preds, labels.long())

tensor(1.6880, grad_fn=<NllLossBackward0>)

In [4]:
input = torch.zeros((5,1,3000), dtype=torch.float32)
net = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, padding="same")
net2 = nn.BatchNorm1d(32)
net2(net(input)).shape

torch.Size([5, 32, 3000])

In [5]:
input = torch.zeros((5,1,3000), dtype=torch.float32)

net = nn.Sequential(
        CNN_block(3000, 1, 32),
        CNN_block(1500, 32, 64),
        CNN_block(750, 64, 64),
        nn.Flatten(),
        nn.Linear(in_features=375*64, out_features=256),
        nn.ReLU(),
        nn.Linear(in_features=256, out_features=10)
    )
net(batch).shape

TypeError: __init__() missing 1 required positional argument: 'out_channels'

In [ ]:

from torchsummary import summary
summary(net, (1, 3000))